SUPPLEMENTARY DATASETS

**BOLD DATASETS ARE RAW DATA, YOU MUST HAVE THOSE TO RUN THE PIPELINE**

**Dataset S1 The characteristics of each patient involved in the study.**

Dataset S2 Summary of laboratory and physical variables before and after treatment in the CMCS treated and placebo groups.

**Dataset S3 Untargeted metabolomics data for each patient before and after treatment.**

Dataset S4 Plasma level of metabolites that are significantly different on Days 14 vs Day 0 in the CMCS and placebo groups. Only metabolites detected in >50% of samples were analyzed. 

Dataset S5 Plasma level of metabolites that are significantly different between the CMCS and placebo groups on Days 0 and Day 14. Only metabolites detected in >50% of samples were analyzed.

Dataset S6 Association between the plasma level all metabolites with the plasma levels of metabolic cofactors including serine, carnitine, nicotinamide riboside, and cysteine as well as degradation products of hydroxychloroquine including hydroquinone sulfate and 2-methoxyhydroquinone sulfate (1).

**Dataset S7 The Olink multiplex inflammation panel used to detect the dynamic range of 72 proteins in plasma samples of the 93 patients participated in the study.**

Dataset S8 Plasma level of inflammation related proteins that are significantly different on Days 14 vs Day 0 in the CMCS and placebo groups. Only proteins detected in >50% of samples were analyzed.

Dataset S9 Plasma level of inflammation related proteins that are significantly different between the CMCS and placebo groups on Days 0 and Day 14. Only metabolites detected in >50% of samples were analyzed. 

Dataset S10 Association between the plasma level all inflammation related proteins with the plasma levels of metabolic cofactors including serine, carnitine, nicotinamide riboside, and cysteine.

Dataset S11 Association between the plasma level of clinical variables including ALT, AST, LDH, Triglycerides (TGs) and hemoglobin with plasma level of all metabolites.

Dataset S12 Association between the plasma level of clinical variables including ALT, AST, LDH, Triglycerides (TGs) and hemoglobin with plasma level of all inflammation related proteins.

Dataset S13 Multi-Omics Network Data, including edges and nodes information. The network is presented in the iNetModels (http://inetmodels.com).

Dataset S14 Association between the plasma level of key inflammation related proteins including MCP-4, IL-18R1, HGF, CXCL10 and CSF-1 with plasma level of all metabolites.


In [ ]:
%matplotlib inline

import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr,ttest_ind,ttest_rel
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
import leidenalg, os, pickle
import igraph as ig

def convert_decimals_and_text(series):
    return series.astype(str).str.replace('<','').str.replace('>','').str.replace('10.000.00','10000').str.replace(',','.').replace('-',np.nan).astype(float)

format_fig = 'pdf'

## Clinical

In [ ]:
temp = pd.read_excel('../ResultsPaper/DS/Altay et al Dataset 1.xlsx', header=1,index_col=0).dropna(how='all')
data_day0 = temp.iloc[0:, (temp.columns.tolist().index('DAY-0')+1):temp.columns.tolist().index('DAY-14')].apply(convert_decimals_and_text)
data_day14 = temp.iloc[0:, (temp.columns.tolist().index('DAY-14')+1):-2].apply(convert_decimals_and_text)
data_day0.index = data_day0.index.astype(int) 
data_day14.index = data_day14.index.astype(int) 
data_day14.columns=data_day0.columns
metadata = temp['GROUP'].str.replace(' ','')

In [ ]:
t_day0 = data_day0.reindex(metadata[metadata == 'CMCS'].index)
p_day0 = data_day0.reindex(metadata[metadata == 'Placebo'].index)
t_day14 = data_day14.reindex(metadata[metadata == 'CMCS'].index)
p_day14 = data_day14.reindex(metadata[metadata == 'Placebo'].index)

res = []

for i in data_day0.columns:
    print(i)
    res.append([i,
                '%.2f ± %.2f' % (p_day0[i].mean(),p_day0[i].std()), 
                '%.2f ± %.2f' % (t_day0[i].mean(),t_day0[i].std()),
                '%.2f ± %.2f' % (p_day14[i].mean(),p_day14[i].std()), 
                '%.2f ± %.2f' % (t_day14[i].mean(),t_day14[i].std()),
                np.log2(t_day0[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/p_day14[i].mean()),
                np.log2(p_day14[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/t_day0[i].mean()),
                
                ttest_ind(t_day0[i],p_day0[i], nan_policy='omit')[1],
                ttest_ind(t_day14[i],p_day14[i], nan_policy='omit')[1],
                ttest_rel(p_day14[i],p_day0[i], nan_policy='omit')[1],
                ttest_rel(t_day14[i],t_day0[i], nan_policy='omit')[1]
               ])

res = pd.DataFrame(res,columns=['Measurement', 
                                'Mean ± SD (Placebo - Day 0)', 
                                'Mean ± SD (Active - Day 0)', 
                                'Mean ± SD (Placebo - Day 14)', 
                                'Mean ± SD (Active - Day 14)',
                                'Log2FoldChange (Active vs Placebo - Day 0)',
                                'Log2FoldChange (Active vs Placebo - Day 14)',
                                'Log2FoldChange (Placebo - Day 14 vs Placebo - Day 0)', 
                                'Log2FoldChange (Active - Day 14 vs Active - Day 0)', 
                                'P-Value (Active vs Placebo - Day 0)',
                                'P-Value (Active vs Placebo - Day 14)',
                                'P-Value (Placebo - Day 14 vs Placebo - Day 0)', 
                                'P-Value (Active - Day 14 vs Active - Day 0)'
                               ]).set_index('Measurement')
res.to_excel('../ResultsPaper/DS/Altay et al Dataset 2.xlsx')

## Metabolomics

In [ ]:
temp = pd.ExcelFile('../ResultsPaper/DS/Altay et al Dataset 3.xlsx')
metadata = temp.parse('Metadata',index_col = 0)
data = temp.parse('Volume & Batch Normalized',index_col = 0).T
mapping = temp.parse('Mapping Metabolites',index_col = 0)
ID_dict = metadata['SUBJECT_ID'].to_dict()

In [ ]:
data_day0 = data.reindex(metadata[metadata['GROUP_ID'].str.contains('v1')].index)
data_day0.index = [ID_dict[i] for i in data_day0.index]
data_day14 = data.reindex(metadata[metadata['GROUP_ID'].str.contains('v2')].index)
data_day14.index = [ID_dict[i] for i in data_day14.index]

In [ ]:
t_day0 = data_day0.reindex(metadata[metadata['CODE'] == 'Active']['SUBJECT_ID'])
p_day0 = data_day0.reindex(metadata[metadata['CODE'] == 'Placebo']['SUBJECT_ID'])
t_day14 = data_day14.reindex(metadata[metadata['CODE'] == 'Active']['SUBJECT_ID'])
p_day14 = data_day14.reindex(metadata[metadata['CODE'] == 'Placebo']['SUBJECT_ID'])

res = []

for i in data_day0.columns:
    res.append([i,
                '%.2f ± %.2f' % (p_day0[i].mean(),p_day0[i].std()), 
                '%.2f ± %.2f' % (t_day0[i].mean(),t_day0[i].std()),
                '%.2f ± %.2f' % (p_day14[i].mean(),p_day14[i].std()), 
                '%.2f ± %.2f' % (t_day14[i].mean(),t_day14[i].std()),
                np.log2(t_day0[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/p_day14[i].mean()),
                np.log2(p_day14[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/t_day0[i].mean()),
                
                ttest_ind(t_day0[i],p_day0[i], nan_policy='omit')[1],
                ttest_ind(t_day14[i],p_day14[i], nan_policy='omit')[1],
                ttest_rel(p_day14[i],p_day0[i], nan_policy='omit')[1],
                ttest_rel(t_day14[i],t_day0[i], nan_policy='omit')[1]
               ])

res = pd.DataFrame(res,columns=['Measurement', 
                                'Mean ± SD (Placebo - Day 0)', 
                                'Mean ± SD (Active - Day 0)', 
                                'Mean ± SD (Placebo - Day 14)', 
                                'Mean ± SD (Active - Day 14)',
                                'Log2FoldChange (Active vs Placebo - Day 0)',
                                'Log2FoldChange (Active vs Placebo - Day 14)',
                                'Log2FoldChange (Placebo - Day 14 vs Placebo - Day 0)', 
                                'Log2FoldChange (Active - Day 14 vs Active - Day 0)', 
                                'P-Value (Active vs Placebo - Day 0)',
                                'P-Value (Active vs Placebo - Day 14)',
                                'P-Value (Placebo - Day 14 vs Placebo - Day 0)', 
                                'P-Value (Active - Day 14 vs Active - Day 0)'
                               ]).set_index('Measurement')
res['FDR (Active vs Placebo - Day 0)']=multipletests(res['P-Value (Active vs Placebo - Day 0)'],method='fdr_bh')[1]
res['FDR (Active vs Placebo - Day 14)']=multipletests(res['P-Value (Active vs Placebo - Day 14)'],method='fdr_bh')[1]
res['FDR (Placebo - Day 14 vs Placebo - Day 0)']=multipletests(res['P-Value (Placebo - Day 14 vs Placebo - Day 0)'],method='fdr_bh')[1]
res['FDR (Active - Day 14 vs Active - Day 0']=multipletests(res['P-Value (Active - Day 14 vs Active - Day 0)'],method='fdr_bh')[1]

pd.concat([res[res.columns[res.columns.str.contains('Mean') | ~res.columns.str.contains('Active vs Placebo')]],mapping.reindex(res.index)],1).set_index('CHEMICAL_NAME').to_excel('../ResultsPaper/DS/Altay et al Dataset 4.xlsx')
pd.concat([res[res.columns[res.columns.str.contains('Mean') | res.columns.str.contains('Active vs Placebo')]],mapping.reindex(res.index)],1).set_index('CHEMICAL_NAME').to_excel('../ResultsPaper/DS/Altay et al Dataset 5.xlsx')

## Proteomics

In [ ]:
temp = pd.ExcelFile('../ResultsPaper/DS/Altay et al Dataset 7.xlsx')
metadata = temp.parse('Metadata',index_col = 0).set_index('SampleName')
data = temp.parse('NPX',index_col = 0)
data_day0 = data.reindex(metadata[metadata['VISIT'].str.contains('Day 0')].index)
data_day14 = data.reindex(metadata[metadata['VISIT'].str.contains('Day 14')].index)

In [ ]:
t_day0 = data_day0.reindex(metadata[metadata['Group'] == 'CMCS'].index).dropna(how='all')
p_day0 = data_day0.reindex(metadata[metadata['Group'] == 'Placebo'].index).dropna(how='all')
t_day14 = data_day14.reindex(metadata[metadata['Group'] == 'CMCS'].index).dropna(how='all')
p_day14 = data_day14.reindex(metadata[metadata['Group'] == 'Placebo'].index).dropna(how='all')
t_day0.index = [i.replace('_day0','') for i in t_day0.index]
p_day0.index = [i.replace('_day0','') for i in p_day0.index]
t_day14.index = [i.replace('_day14','') for i in t_day14.index]
p_day14.index = [i.replace('_day14','') for i in p_day14.index]


res = []

for i in data_day0.columns:
    res.append([i,
                '%.2f ± %.2f' % (p_day0[i].mean(),p_day0[i].std()), 
                '%.2f ± %.2f' % (t_day0[i].mean(),t_day0[i].std()),
                '%.2f ± %.2f' % (p_day14[i].mean(),p_day14[i].std()), 
                '%.2f ± %.2f' % (t_day14[i].mean(),t_day14[i].std()),
                np.log2(t_day0[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/p_day14[i].mean()),
                np.log2(p_day14[i].mean()/p_day0[i].mean()),
                np.log2(t_day14[i].mean()/t_day0[i].mean()),
                
                ttest_ind(t_day0[i],p_day0[i], nan_policy='omit')[1],
                ttest_ind(t_day14[i],p_day14[i], nan_policy='omit')[1],
                ttest_rel(p_day14[i],p_day0[i], nan_policy='omit')[1],
                ttest_rel(t_day14[i],t_day0[i], nan_policy='omit')[1]
               ])

res = pd.DataFrame(res,columns=['Measurement', 
                                'Mean ± SD (Placebo - Day 0)', 
                                'Mean ± SD (Active - Day 0)', 
                                'Mean ± SD (Placebo - Day 14)', 
                                'Mean ± SD (Active - Day 14)',
                                'Log2FoldChange (Active vs Placebo - Day 0)',
                                'Log2FoldChange (Active vs Placebo - Day 14)',
                                'Log2FoldChange (Placebo - Day 14 vs Placebo - Day 0)', 
                                'Log2FoldChange (Active - Day 14 vs Active - Day 0)', 
                                'P-Value (Active vs Placebo - Day 0)',
                                'P-Value (Active vs Placebo - Day 14)',
                                'P-Value (Placebo - Day 14 vs Placebo - Day 0)', 
                                'P-Value (Active - Day 14 vs Active - Day 0)'
                               ]).set_index('Measurement')
res['FDR (Active vs Placebo - Day 0)']=multipletests(res['P-Value (Active vs Placebo - Day 0)'],method='fdr_bh')[1]
res['FDR (Active vs Placebo - Day 14)']=multipletests(res['P-Value (Active vs Placebo - Day 14)'],method='fdr_bh')[1]
res['FDR (Placebo - Day 14 vs Placebo - Day 0)']=multipletests(res['P-Value (Placebo - Day 14 vs Placebo - Day 0)'],method='fdr_bh')[1]
res['FDR (Active - Day 14 vs Active - Day 0']=multipletests(res['P-Value (Active - Day 14 vs Active - Day 0)'],method='fdr_bh')[1]



res[res.columns[res.columns.str.contains('Mean') | ~res.columns.str.contains('Active vs Placebo')]].to_excel('../ResultsPaper/DS/Altay et al Dataset 8.xlsx')
res[res.columns[res.columns.str.contains('Mean') | res.columns.str.contains('Active vs Placebo')]].to_excel('../ResultsPaper/DS/Altay et al Dataset 9.xlsx')

## Data Integration

In [ ]:
class Network_Analysis:
    def __init__(self,raw_data,nodes,respath):
        self.res_path=respath
        self.writer = pd.ExcelWriter('%s/Altay et al Dataset 13.xlsx' % self.res_path, engine='xlsxwriter')
        self.network_ori=self.__calc(raw_data)
        self.nodes=nodes
        print('Network Analysis')
        self.__net_analysis_combi()
        self.writer.save()
    
    def __calc(self,df):
        print('Calculating Correlation..')
        temp=spearmanr(df.T, nan_policy='omit')
        corr=pd.DataFrame(temp[0],columns=list(df.index),index=list(df.index))
        pval=pd.DataFrame(temp[1],columns=list(df.index),index=list(df.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        res=res[['level_0','level_1','weight','pval']]
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res.columns=['source','target','correlation','pvalue']
        res.to_excel(self.writer, sheet_name='Edges (Unfiltered)', index = False)
        res=res[res.padj < 0.05].reset_index(drop=True)
        res.columns=['source','target','correlation','pvalue','padj']
        res=res[['source','target','correlation','pvalue','padj']]
        print('Done!!')
        return res
    
    def __net_analysis_combi(self):
        print('Loading The Network...')
        temp=self.network_ori
        g= ig.Graph.TupleList(zip(temp['source'],temp['target'],temp['correlation']),weights=True)
        self.network = g
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        self.clustering_combi=pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        print('Cluster Analysis...')
        self.modularity_combi=diff
        self.nodes['cluster'] = self.clustering_combi.reindex(self.nodes.index).tolist()
        self.nodes.to_excel(self.writer, sheet_name='Nodes')
    
    def save_network(self):
        print('Saving The Network..')
        pickle_out = open('%s/network_object.pkl' % self.res_path,"wb")
        self.writer = None
        pickle.dump(self, pickle_out)
        pickle_out.close()

In [ ]:
temp = pd.read_excel('../ResultsPaper/DS/Altay et al Dataset 1.xlsx', header=1,index_col=0).dropna(how='all')
data_day0 = temp.iloc[0:, (temp.columns.tolist().index('DAY-0')+1):temp.columns.tolist().index('DAY-14')].apply(convert_decimals_and_text)
data_day14 = temp.iloc[0:, (temp.columns.tolist().index('DAY-14')+1):-2].apply(convert_decimals_and_text)
data_day0.index = ['Patient_'+str(i)+'_day0' for i in data_day0.index.astype(int) ]
data_day14.index = ['Patient_'+str(i)+'_day14' for i in data_day14.index.astype(int) ]
data_day14.columns=data_day0.columns
data_clin = pd.concat([data_day0,data_day14]).T

temp = pd.ExcelFile('../ResultsPaper/DS/Altay et al Dataset 7.xlsx')
data_prots = temp.parse('NPX',index_col = 0).T

temp = pd.ExcelFile('../ResultsPaper/DS/Altay et al Dataset 3.xlsx')
metadata_mets = temp.parse('Metadata',index_col = 0)
metadata_mets['name'] = ('Patient'+'_'+metadata_mets['SUBJECT_ID'].astype(str)+'_'+metadata_mets['VISIT']).str.replace('v1','day0').replace('v2','day14')
data_mets = temp.parse('Volume & Batch Normalized',index_col = 0)
mapping_mets = temp.parse('Mapping Metabolites',index_col = 0)
ID_dict = metadata_mets['name'].to_dict()
data_mets.columns = [ID_dict[i] for i in data_mets.columns]

data=pd.concat([data_mets,data_prots,data_clin])


In [ ]:
nodes_mets = mapping_mets.copy()
nodes_mets['LOC'] = 'METABOLITE'
nodes_prots = pd.DataFrame(pd.Series(data_prots.index, index = data_prots.index, name = "CHEMICAL_NAME"))
nodes_prots['LOC'] = 'PROTEIN'
nodes_clin = pd.DataFrame(pd.Series(data_clin.index, index = data_clin.index, name = "CHEMICAL_NAME"))
nodes_clin['LOC'] = 'CLINICAL'
nodes = pd.concat([nodes_mets,nodes_prots,nodes_clin])

In [ ]:
############
#AVOID RUNNING THIS UNLESS NECESSARY
############

# print('StartingNet')

# k=Network_Analysis(raw_data=data,nodes=nodes,respath='../ResultsPaper/DS/')
# k.save_network()

## Correlation with Specific Analytes

In [ ]:
temp = pd.ExcelFile('../ResultsPaper/DS/Altay et al Dataset 13.xlsx')
edges = temp.parse('Edges (Unfiltered)')
nodes = temp.parse('Nodes',index_col = 0)

In [ ]:
selected_mets = ['serine','carnitine','nicotinamide riboside','cysteine','2-methoxyhydroquinone sulfate (1)','hydroquinone sulfate']
selected_clin = ['Alanine aminotransferase (U/L)','Aspartate transaminase (U/L)','Lactate dehydrogenase (U/L)','Triglyceride (mg/dL)','Hemoglobin (g/dL)']
#prelim prots they are significant in active day 14 vs 0, and also within top 10 most central proteins in the network
selected_prots = ['CXCL10','CSF-1','IL-18R1','MCP-4','HGF']

In [ ]:
name_col = 'CHEMICAL_NAME'
neigh = 'METABOLITE'
nodes_mapping = nodes[name_col].to_dict()
temp_metn = nodes[nodes[name_col].isin(selected_mets)]
temp_mete = {}
for i in temp_metn.index:
    name = nodes_mapping[i]
    temp1 = edges[(edges['source'] == i) & edges['target'].isin(nodes[nodes['LOC'] == neigh].index)][['target','correlation','pval']]
    temp1.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp2 = edges[(edges['target'] == i) & edges['source'].isin(nodes[nodes['LOC'] == neigh].index)][['source','correlation','pval']]
    temp2.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp_fin = pd.concat([temp1,temp2])
    temp_fin['Metabolites'] = temp_fin['Metabolites'].replace(nodes_mapping)
    temp_fin['FDR']=multipletests(temp_fin['P-Value'],method='fdr_bh')[1]
    temp_mete[name] = temp_fin.set_index('Metabolites')
pd.concat(temp_mete,keys=temp_mete.keys(),axis = 1).to_excel('../ResultsPaper/DS/Altay et al Dataset 6.xlsx')

In [ ]:
name_col = 'CHEMICAL_NAME'
neigh = 'PROTEIN'
nodes_mapping = nodes[name_col].to_dict()
temp_met = nodes[nodes[name_col].isin(selected_mets[0:4])]
temp_mete = {}
for i in temp_met.index:
    name = nodes_mapping[i]
    temp1 = edges[(edges['source'] == i) & edges['target'].isin(nodes[nodes['LOC'] == neigh].index)][['target','correlation','pval']]
    temp1.columns = ['Proteins', 'Correlation', 'P-Value']
    temp2 = edges[(edges['target'] == i) & edges['source'].isin(nodes[nodes['LOC'] == neigh].index)][['source','correlation','pval']]
    temp2.columns = ['Proteins', 'Correlation', 'P-Value']
    temp_fin = pd.concat([temp1,temp2])
    temp_fin['Proteins'] = temp_fin['Proteins'].replace(nodes_mapping)
#     temp_fin['FDR']=multipletests(temp_fin['P-Value'],method='fdr_bh')[1]
    temp_mete[name] = temp_fin.set_index('Proteins')
pd.concat(temp_mete,keys=temp_mete.keys(),axis = 1).to_excel('../ResultsPaper/DS/Altay et al Dataset 10.xlsx')

In [ ]:
name_col = 'CHEMICAL_NAME'
neigh = 'METABOLITE'
nodes_mapping = nodes[name_col].to_dict()
temp_met = nodes[nodes[name_col].isin(selected_clin)]
temp_mete = {}
for i in temp_met.index:
    name = nodes_mapping[i]
    temp1 = edges[(edges['source'] == i) & edges['target'].isin(nodes[nodes['LOC'] == neigh].index)][['target','correlation','pval']]
    temp1.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp2 = edges[(edges['target'] == i) & edges['source'].isin(nodes[nodes['LOC'] == neigh].index)][['source','correlation','pval']]
    temp2.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp_fin = pd.concat([temp1,temp2])
    temp_fin['Metabolites'] = temp_fin['Metabolites'].replace(nodes_mapping)
    temp_fin['FDR']=multipletests(temp_fin['P-Value'],method='fdr_bh')[1]
    temp_mete[name] = temp_fin.set_index('Metabolites')
pd.concat(temp_mete,keys=temp_mete.keys(),axis = 1).to_excel('../ResultsPaper/DS/Altay et al Dataset 11.xlsx')

In [ ]:
name_col = 'CHEMICAL_NAME'
neigh = 'PROTEIN'
nodes_mapping = nodes[name_col].to_dict()
temp_met = nodes[nodes[name_col].isin(selected_clin)]
temp_mete = {}
for i in temp_met.index:
    name = nodes_mapping[i]
    temp1 = edges[(edges['source'] == i) & edges['target'].isin(nodes[nodes['LOC'] == neigh].index)][['target','correlation','pval']]
    temp1.columns = ['Proteins', 'Correlation', 'P-Value']
    temp2 = edges[(edges['target'] == i) & edges['source'].isin(nodes[nodes['LOC'] == neigh].index)][['source','correlation','pval']]
    temp2.columns = ['Proteins', 'Correlation', 'P-Value']
    temp_fin = pd.concat([temp1,temp2])
    temp_fin['Proteins'] = temp_fin['Proteins'].replace(nodes_mapping)
    temp_fin['FDR']=multipletests(temp_fin['P-Value'],method='fdr_bh')[1]
    temp_mete[name] = temp_fin.set_index('Proteins')
pd.concat(temp_mete,keys=temp_mete.keys(),axis = 1).to_excel('../ResultsPaper/DS/Altay et al Dataset 12.xlsx')

In [ ]:
name_col = 'CHEMICAL_NAME'
neigh = 'METABOLITE'
nodes_mapping = nodes[name_col].to_dict()
temp_met = nodes[nodes[name_col].isin(selected_prots)]
temp_mete = {}
for i in temp_met.index:
    name = nodes_mapping[i]
    temp1 = edges[(edges['source'] == i) & edges['target'].isin(nodes[nodes['LOC'] == neigh].index)][['target','correlation','pval']]
    temp1.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp2 = edges[(edges['target'] == i) & edges['source'].isin(nodes[nodes['LOC'] == neigh].index)][['source','correlation','pval']]
    temp2.columns = ['Metabolites', 'Correlation', 'P-Value']
    temp_fin = pd.concat([temp1,temp2])
    temp_fin['Metabolites'] = temp_fin['Metabolites'].replace(nodes_mapping)
    temp_fin['FDR']=multipletests(temp_fin['P-Value'],method='fdr_bh')[1]
    temp_mete[name] = temp_fin.set_index('Metabolites')
pd.concat(temp_mete,keys=temp_mete.keys(),axis = 1).to_excel('../ResultsPaper/DS/Altay et al Dataset 14.xlsx')